In [130]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors
import pickle
import random
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [26]:
random.seed(3214)

In [27]:
with open('../data/pickles/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [28]:
df = pd.read_csv('../data/csv/merge_df.csv', index_col=0)
df.head()

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,0.008984,0.031454,0.108398,0.029397,0.009277,0.014918
1,061172.mp3,-692.448303,117.801537,18.185888,5.736947,5.938968,31.851004,1.060427,4.214683,-0.146251,...,-0.198049,0.948205,0.390652,0.080923,0.000357,0.059621,0.351562,0.068662,0.002930,0.058827
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,0.017999,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,0.022331,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,0.010698,0.055768,0.450195,0.098991,0.005859,0.067057


In [29]:
test_idx = list()
while len(test_idx) < 100:
    idx = random.randint(0, df.shape[0] - 1)
    track_id = int(df.loc[idx].track[:-4])
    tags = main_dict[track_id]['all_tags']
    if len(tags) >= 10:
        test_idx.append(idx)

In [30]:
df_test = df.loc[test_idx]
df_test

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
7301,010673.mp3,-626.681091,132.151154,11.753740,45.711430,-0.209040,10.068515,-0.682384,5.395576,-4.435939,...,0.461337,0.281160,0.329500,0.124018,0.027167,0.045374,0.130859,0.031217,0.003418,0.019365
3931,066643.mp3,-687.945374,141.427277,-16.972485,19.092112,6.906140,3.117708,6.967309,-2.045398,-7.208217,...,-0.111801,0.627290,0.380662,0.080697,0.000165,0.061523,0.282227,0.049509,0.002930,0.025951
5176,000194.mp3,-782.241943,178.044266,-29.330959,18.219608,-14.123279,-10.458302,-16.387886,-5.279102,-16.686710,...,0.276664,0.634153,0.264619,0.043619,0.000129,0.038136,0.169922,0.037896,0.001953,0.017058
80,074421.mp3,-623.853394,122.672737,17.257471,4.505549,0.719744,-2.922961,-5.741470,-1.727011,-18.558123,...,0.421106,0.539590,0.264868,0.082399,0.007031,0.040712,0.224121,0.051835,0.009277,0.032241
7903,131553.mp3,-645.512512,74.718880,12.483846,45.145756,38.911320,16.187067,6.199753,0.013115,-3.431449,...,0.100123,0.391042,0.335317,0.112432,0.000625,0.050604,0.271973,0.079668,0.008789,0.049538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708,025603.mp3,-705.887451,168.620865,34.891018,33.937485,15.525203,2.845042,-8.320030,2.128073,10.097413,...,0.422947,1.021024,0.493078,0.113007,0.003961,0.076067,0.147949,0.026009,0.002441,0.020538
3319,075417.mp3,-637.247192,73.643715,17.631123,33.222134,11.122272,9.398943,12.663881,11.582001,10.201411,...,0.267713,0.341603,0.467992,0.120776,0.001821,0.087565,0.348633,0.081037,0.009766,0.044586
5911,125154.mp3,-694.168884,92.781448,59.304184,42.092861,12.254707,19.374973,4.323670,7.782975,-5.282559,...,-0.067733,0.774724,0.482886,0.110169,0.003097,0.075931,0.317871,0.064865,0.002441,0.054606
3341,075436.mp3,-800.051819,241.068558,40.636051,-0.057632,6.397528,-18.648108,-1.818632,3.204657,-0.186070,...,0.923011,1.546881,1.210505,0.112725,0.010022,0.145565,0.138184,0.015291,0.000488,0.012997


In [98]:
df_train = df.loc[list(set(df.index.values) - set(test_idx))]
df_train

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,8.983906e-03,0.031454,0.108398,0.029397,0.009277,0.014918
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,1.799921e-02,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,2.233087e-02,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,1.069842e-02,0.055768,0.450195,0.098991,0.005859,0.067057
5,061592.mp3,-708.504639,136.741913,9.762825,40.804737,17.343578,28.947464,15.307942,17.034895,9.786160,...,-0.039187,0.629073,0.458335,0.132431,6.568079e-11,0.090427,0.515137,0.018989,0.002441,0.039541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,131914.mp3,-654.926941,144.197983,-5.618096,12.815182,-4.743158,4.794647,-0.457318,2.406655,-8.439367,...,0.907090,0.585478,0.172700,0.063363,1.671647e-02,0.027547,0.136230,0.058892,0.012207,0.026086
7978,131772.mp3,-625.237183,158.683304,-10.997794,23.054964,11.636267,-1.839731,-17.972336,-9.967291,-12.986640,...,0.608825,0.747877,0.245719,0.111260,6.102545e-03,0.040364,0.070312,0.020365,0.005859,0.009048
7979,131792.mp3,-655.022644,144.006958,5.970843,14.379925,14.236056,3.561646,-4.041969,-4.800819,-3.666181,...,0.740162,0.707120,0.290256,0.078840,7.514172e-03,0.051776,0.104980,0.032318,0.000488,0.014633
7980,131624.mp3,-788.494507,75.723251,35.248608,28.392780,4.289547,18.946749,-5.460835,10.575089,-8.978883,...,0.130618,0.542250,0.212018,0.045943,9.695412e-04,0.043634,0.321289,0.102740,0.007812,0.053932


In [85]:
features_train = df_train.drop(['track'], 1)
features_test = df_test.drop(['track'], 1)

In [86]:
scaler = StandardScaler()
scaler.fit(features_train)
X_train = scaler.transform(features_train)
X_test = scaler.transform(features_test)

In [87]:
pca = PCA(0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [88]:
X_train.shape

(7885, 103)

In [89]:
X_test.shape

(100, 103)

In [32]:
result = list()
for track in df_train.track.values:
    idx = int(track[:-4])
    if len(main_dict[idx]['all_tags']) >=10:
        result.append(track)
        

In [33]:
len(result)

4016

In [50]:
similarity_1 = list()
similarity_2 = list()
similarity_3 = list()
similarity_4 = list()
similarity_5 = list()
similarity_6 = list()
similarity_7 = list()
similarity_8 = list()
similarity_9 = list()


In [71]:
no_similarity = list()
# while len(similarity_9) < 1000:
for i in range(1000000):
    track_1, track_2 = random.sample(result, 2)
    tags_1 = main_dict[int(track_1[:-4])]['all_tags']
    tags_2 = main_dict[int(track_2[:-4])]['all_tags']
    intersection = set(tags_1) & set(tags_2)
    if len(intersection) == 0:
#         similarity_1.append((track_1, track_2, 0))
        continue
    precision = len(intersection) / len(tags_1)
    recall = len(intersection) / len(tags_2)
    score = 2 * precision * recall / (precision + recall)
    score = round(score, 2)
    if 0.8 <= score < 0.9:
        similarity_9.append((track_1, track_2, score))


In [74]:
all_similarity = similarity_1 +\
similarity_2 +\
similarity_3 +\
similarity_4 +\
similarity_5 +\
similarity_6 +\
similarity_7 +\
similarity_8 +\
similarity_9

In [77]:
all_similarity = list(set(all_similarity))

In [78]:
len(all_similarity)

8231

In [80]:
for tuple in all_similarity:
    with open('../data/tags_similarity.txt', 'a+') as f:
        f.write(f'{tuple[0]} {tuple[1]} {tuple[2]}' + '\n')

In [107]:
df_train_2 = df_train.reset_index(drop=True)
# df_train_2.reset_index(drop=True, inplace=True)
df_train_2

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,8.983906e-03,0.031454,0.108398,0.029397,0.009277,0.014918
1,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,1.799921e-02,0.079505,0.125000,0.028202,0.000977,0.020876
2,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,2.233087e-02,0.065322,0.267090,0.083095,0.006348,0.036801
3,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,1.069842e-02,0.055768,0.450195,0.098991,0.005859,0.067057
4,061592.mp3,-708.504639,136.741913,9.762825,40.804737,17.343578,28.947464,15.307942,17.034895,9.786160,...,-0.039187,0.629073,0.458335,0.132431,6.568079e-11,0.090427,0.515137,0.018989,0.002441,0.039541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,131914.mp3,-654.926941,144.197983,-5.618096,12.815182,-4.743158,4.794647,-0.457318,2.406655,-8.439367,...,0.907090,0.585478,0.172700,0.063363,1.671647e-02,0.027547,0.136230,0.058892,0.012207,0.026086
7881,131772.mp3,-625.237183,158.683304,-10.997794,23.054964,11.636267,-1.839731,-17.972336,-9.967291,-12.986640,...,0.608825,0.747877,0.245719,0.111260,6.102545e-03,0.040364,0.070312,0.020365,0.005859,0.009048
7882,131792.mp3,-655.022644,144.006958,5.970843,14.379925,14.236056,3.561646,-4.041969,-4.800819,-3.666181,...,0.740162,0.707120,0.290256,0.078840,7.514172e-03,0.051776,0.104980,0.032318,0.000488,0.014633
7883,131624.mp3,-788.494507,75.723251,35.248608,28.392780,4.289547,18.946749,-5.460835,10.575089,-8.978883,...,0.130618,0.542250,0.212018,0.045943,9.695412e-04,0.043634,0.321289,0.102740,0.007812,0.053932


In [116]:
features = list()
target = list()
for tupple in all_similarity:
    idx_1 = df_train_2[df_train_2.track == tupple[0]].index[0]
    idx_2 = df_train_2[df_train_2.track == tupple[1]].index[0]
    arr_1 = X_train[idx_1]
    arr_2 = X_train[idx_2]
    combo = np.hstack((arr_1, arr_2))
    features.append(combo)
    target.append(tupple[2])

In [124]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.1, 
    random_state=123
)

In [126]:
gbr = GradientBoostingRegressor()

In [128]:
gbr.fit(features_train, target_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [129]:
target_pred = gbr.predict(features_test)

In [134]:
target_train_pred = gbr.predict(features_trainn)

In [132]:
mean_squared_error(target_test, target_pred)

0.05004046830651699

In [133]:
r2_score(target_test, target_pred)

0.18272335005669849

In [135]:
mean_squared_error(target_train, target_train_pred)

0.041178381138963584

In [136]:
r2_score(target_train, target_train_pred)

0.31528048184053437

In [146]:
target_test[432]

0.6

In [147]:
target_pred[432]

0.5337232892568152

In [215]:
neigh = NearestNeighbors(20)
neigh.fit(X_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [216]:
def count_good_recommend(query_idx, answer_list, trhold):
    counter = 0
    query_id = int(df_test.iloc[query_idx].track[:-4])
    relevant = [query_id]
    query_tags = main_dict[query_id]['all_tags']
    for idx in answer_list:
        answer_id = int(df_train.iloc[idx].track[:-4])
        answer_tags = main_dict[answer_id]['all_tags']
        score = 0
        i = 0
        for tag in answer_tags:
            if tag in query_tags:
                i += 1
        if i == 0:
            continue
        
        precision = i / len(answer_tags)
        recall = i / len(query_tags)
        score = 2 * precision * recall / (precision + recall)
        if score >= trhold:        
            counter += 1
            relevant.append(answer_id)
    return counter, relevant

In [217]:
def count_ndcg(neighbors, relevant_list):
    dcg = 0
    idcg = 0
    for i in range(len(neighbors)):
        track = int(df_train.iloc[neighbors[i]].track[:-4])
        if track in relevant_list:
            dcg += 1 / math.log(i + 2, 2)
        idcg += 1 / math.log(i + 2, 2)
    ndcg = dcg / idcg
    return ndcg

In [218]:
def rank_by_model(model, neighb, arr_test):
    result = list()
    for idx in neighb:
        arr_2 = X_train[idx]
        combo = np.hstack((arr_test, arr_2))
        score = model.predict([combo])
        result.append((idx, score))
    top_10 = sorted(result, key=lambda x: x[1], reverse=True)[:10]
    top_10 = [x[0] for x in top_10]

    return top_10

In [222]:
mean_ap = 0
ndcg = 0
for i in range(100):
    neighbors = neigh.kneighbors([X_test[i]])[1][0]
    level_2 = rank_by_model(gbr, neighbors, X_test[i])
    relevant_num, relevant_list = count_good_recommend(i, level_2, trhold=0.4)
#     print(relevant_list)
    print(relevant_list[0], [df_train.iloc[x].track[:-4] for x in level_2])
    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(level_2, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

10673 ['141874', '094034', '042761', '111378', '106456', '064536', '030682', '047194', '091851', '126602']
66643 ['133781', '122087', '098301', '091329', '145260', '091468', '085426', '129395', '142575', '057164']
194 ['043590', '012481', '006379', '078213', '000211', '004069', '125185', '016744', '006331', '038779']
74421 ['066539', '134790', '004076', '097586', '106458', '064247', '052128', '085490', '148214', '130944']
131553 ['122080', '042475', '055831', '126295', '004073', '019439', '099260', '108809', '126104', '055828']
24431 ['056804', '024425', '024422', '118070', '081555', '116491', '030316', '149141', '052141', '129874']
144469 ['127037', '136995', '117887', '095724', '003273', '122079', '126455', '084139', '088860', '067017']
87100 ['108023', '093917', '010439', '087098', '133973', '082918', '116753', '121655', '126241', '019417']
69204 ['123645', '085318', '006439', '035549', '150078', '122808', '152103', '085310', '085307', '085596']
27454 ['122683', '124891', '020296', 

40851 ['068531', '106277', '075755', '085828', '064334', '075937', '145257', '114293', '128927', '085823']
1686 ['131935', '004076', '006368', '004078', '006390', '127263', '132138', '089857', '006358', '004234']
73365 ['081576', '122683', '069822', '124891', '120314', '120304', '026655', '085311', '073371', '042914']
5264 ['053937', '075427', '023041', '142949', '073770', '051271', '060871', '019891', '089447', '121590']
19759 ['006448', '134934', '128448', '062193', '098299', '075194', '019758', '081083', '143221', '036277']
41568 ['106339', '068897', '107567', '034003', '081613', '122936', '054235', '096696', '121738', '105714']
24429 ['122628', '110650', '067457', '052449', '113789', '097960', '069784', '042914', '040239', '082932']
86788 ['073560', '041825', '138064', '083790', '111306', '096403', '096399', '109350', '069193', '087153']
152418 ['140043', '060536', '038776', '010678', '093986', '024702', '124517', '060547', '127559', '122398']
64630 ['126103', '138578', '130456', '

In [220]:
mean_ap

0.07699999999999997

In [221]:
mean_ndcg

0.07371165059477855

In [81]:
all_similarity[0]

('081512.mp3', '081781.mp3', 0.79)

In [84]:
df_train[df_train.track == '081512.mp3'].drop(columns=['track']).values

array([[-7.74647705e+02,  1.26531204e+02,  1.43112164e+01,
         1.14453869e+01, -8.95931244e+00, -9.55889320e+00,
        -1.35218897e+01,  2.28147745e+00,  1.78788033e+01,
         7.60553885e+00,  6.46068764e+00,  5.74957466e+00,
         2.95985079e+00,  4.32843555e+03,  1.02788281e+03,
         1.08554832e+02,  9.14352112e+02,  1.19996819e+02,
         1.40324879e+01, -1.35242832e+00,  1.88848228e+01,
         1.13695312e+04,  1.68837732e+03,  8.61328125e+01,
         2.64714722e+03,  9.10147858e+00,  2.54628015e+00,
         4.16384824e-02,  1.58443892e+00,  3.64557415e-01,
         7.18303695e-02,  6.19487313e-04,  7.85310268e-02,
         3.59863281e-01,  5.66855259e-02,  2.44140625e-03,
         6.01763166e-02, -8.02175903e+02,  1.36923691e+02,
         2.09778728e+01,  2.03402443e+01,  2.40463424e+00,
         6.60363150e+00,  4.70998526e+00,  1.70874729e+01,
         2.93335152e+01,  1.52756615e+01,  3.09140229e+00,
        -4.18995047e+00, -4.08248231e-03,  3.83288086e+0

In [39]:
a = main_dict[84096]['all_tags']
b = main_dict[136994]['all_tags']
set(a) & set(a)

{'abstract',
 'abstract hip hop',
 'braindance',
 'breakbeat',
 'breakcore',
 'drum & bass',
 'dubstep',
 'electronic',
 'electronica',
 'glitch',
 'idm',
 'under 2000 listeners'}